In [1]:
#### loading the dataset #####

!wget -nc https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv

--2023-05-06 18:38:06--  https://lazyprogrammer.me/course_files/nlp/bbc_text_cls.csv
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.21.23.210, 172.67.213.166, 2606:4700:3030::ac43:d5a6, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.21.23.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5085081 (4.8M) [text/csv]
Saving to: ‘bbc_text_cls.csv’

bbc_text_cls.csv    100%[===================>]   4.85M  --.-KB/s    in 0.08s   

2023-05-06 18:38:06 (59.5 MB/s) - ‘bbc_text_cls.csv’ saved [5085081/5085081]



In [63]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, Input
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import numpy as np

In [64]:
### reading the csv file #####

bbc_text = pd.read_csv('bbc_text_cls.csv')

In [65]:
bbc_text.head()

,text,labels
0,Ad sales boost Time Warner profit\n\nQuarterly...,business
1,Dollar gains on Greenspan speech\n\nThe dollar...,business
2,Yukos unit buyer faces loan claim\n\nThe owner...,business
3,High fuel prices hit BA's profits\n\nBritish A...,business
4,Pernod takeover talk lifts Domecq\n\nShares in...,business


In [66]:
#### different labels available ######

set(bbc_text['labels'])

{'business', 'entertainment', 'politics', 'sport', 'tech'}

In [67]:
##### selecting only sport news since we want topic specific suggestions #####

sport_df = bbc_text[bbc_text['labels'] == 'sport']

In [68]:
sport_df.head()

,text,labels
1313,Claxton hunting first major medal\n\nBritish h...,sport
1314,O'Sullivan could run in Worlds\n\nSonia O'Sull...,sport
1315,Greene sets sights on world title\n\nMaurice G...,sport
1316,IAAF launches fight against drugs\n\nThe IAAF ...,sport
1317,"Dibaba breaks 5,000m world record\n\nEthiopia'...",sport


In [69]:
corpus = sport_df['text']

In [71]:
corpus = corpus[0:100] ### selecting first 100 sentences for faster training ######

In [72]:
# Define the vocabulary
vocab = set()
for sentence in corpus:
    for word in sentence.split():
        vocab.add(word)
vocab_size = len(vocab)

In [73]:
# Tokenize the corpus

tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)

In [74]:
# Set hyperparameters

embedding_dim = 100
window_size = 2
num_epochs = 100

In [75]:
# Generate word pairs as input-output pairs for skip-gram
word_pairs = []
for sequence in sequences:
    for i in range(len(sequence)):
        for j in range(max(0, i - window_size), min(i + window_size + 1, len(sequence))):
            if i != j:
                word_pairs.append((sequence[i], sequence[j]))

In [76]:
# Prepare input and output data

input_data, output_data = zip(*word_pairs)
input_data = np.array(input_data)
output_data = np.array(output_data)


In [77]:
# Define the model architecture
input_layer = Input(shape=(1,))
embedding_layer = Embedding(vocab_size, embedding_dim)(input_layer)
output_layer = Dense(vocab_size, activation='softmax')(embedding_layer)

### using softmax as it is the standard algorithm to convert a input of K numbers into a probability distribution of K possible outcomes, which is what we need for this exercise ####


In [78]:
## Define the model ##

model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [79]:
# Train the model
model.fit(input_data, output_data, epochs=num_epochs)

Train on 128304 samples
Epoch 1/100
128304/128304 [==============================] - 68s 529us/sample - loss: 6.9126
Epoch 2/100
128304/128304 [==============================] - 67s 520us/sample - loss: 6.3699
Epoch 3/100
128304/128304 [==============================] - 66s 516us/sample - loss: 6.0972
Epoch 4/100
128304/128304 [==============================] - 66s 515us/sample - loss: 5.8462
Epoch 5/100
128304/128304 [==============================] - 67s 520us/sample - loss: 5.6255
Epoch 6/100
128304/128304 [==============================] - 68s 528us/sample - loss: 5.4356
Epoch 7/100
128304/128304 [==============================] - 67s 521us/sample - loss: 5.2719
Epoch 8/100
128304/128304 [==============================] - 68s 526us/sample - loss: 5.1323
Epoch 9/100
128304/128304 [==============================] - 68s 526us/sample - loss: 5.0152
Epoch 10/100
128304/128304 [==============================] - 67s 523us/sample - loss: 4.9178
Epoch 11/100
128304/128304 [=================

In [80]:
# Get the learned word embeddings

weights = model.layers[1].get_weights()[0]

In [ ]:

# Print the word embeddings
word_index = tokenizer.word_index
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if i < vocab_size:
        embedding_matrix[i] = weights[i]

print("Word Embeddings:")
for word, i in word_index.items():
    if i < vocab_size:
        print(word, embedding_matrix[i])

In [94]:
# Define a function to find the nearest words to a given word
def find_nearest_words(word, embeddings, vocab, n=5):
    # Get the index of the word in the vocabulary
    word_index = vocab.index(word)
    # Get the word embedding
    word_embedding = embeddings[word_index]
    # Compute the cosine similarity between the word embedding and all other word embeddings
    cosine_similarities = np.dot(embeddings, word_embedding) / (np.linalg.norm(embeddings, axis=1) * np.linalg.norm(word_embedding))
    # Get the indices of the most similar words
    nearest_indices = cosine_similarities.argsort()[-n-1:-1][::-1]
    # Get the most similar words
    nearest_words = [vocab[i] for i in nearest_indices]
    return nearest_words

# Find the nearest words to the word 'sentence'
nearest_words = find_nearest_words('breaks', embedding_matrix, list(vocab))
print("Nearest words to 'sentence':", nearest_words)

Nearest words to 'sentence': ['19-year-old.', 'passing', 'predict', '3:49.78.', 'who']


<ipython-input-94-f6d7bceb36ea>:8: RuntimeWarning: invalid value encountered in true_divide
  cosine_similarities = np.dot(embeddings, word_embedding) / (np.linalg.norm(embeddings, axis=1) * np.linalg.norm(word_embedding))


In [ ]:
corpus

In [ ]:
##### using pre-trained embeddings ######

In [89]:
import gensim.downloader as api

# Load pre-trained word embeddings
embedding_model = api.load("glove-wiki-gigaword-100")



[================================================--] 96.7% 123.8/128.1MB downloaded


In [93]:
word = "breaks"
nearest_words = embedding_model.most_similar(word)

# Print the nearest words
print("Nearest words to '{}':".format(word))
for word, similarity in nearest_words:
    print(word, similarity)

Nearest words to 'breaks':
break 0.714804470539093
gets 0.6286557912826538
breaking 0.624260425567627
goes 0.6123769283294678
cut 0.6074079871177673
cuts 0.6058018803596497
takes 0.6039687991142273
puts 0.5913140773773193
money 0.5860028862953186
gives 0.5848631858825684
